<a href="https://colab.research.google.com/github/Kudston/llms_example_projects/blob/main/Writing_a_custom_preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade keras_nlp
!pip install -q --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import tensorflow as tf

In [ ]:
import os

# os.environ("KERAS_BACKEND") = "jax"

import keras
import keras_nlp

keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  42.0M      0  0:00:01  0:00:01 --:--:-- 42.0M


In [ ]:
train_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=32, validation_split=0.2, subset="training", seed=42
)
test_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=32, validation_split=0.2, subset="validation", seed=42
)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
print(train_dataset.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(), dtype=string, numpy=b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


CREATE A CUSTOM PREPROCESSOR

In [ ]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset('bert_tiny_en_uncased', sequence_length=512)
backbone  = keras_nlp.models.BertBackbone.from_preset('bert_tiny_en_uncased')

imdb_train_ds = train_dataset.map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
imdb_test_ds = test_dataset.map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

backbone.trainable = False
inputs = backbone.input
sequence = backbone(inputs)['sequence_output']
for _ in range(2):
    sequence = keras_nlp.layers.TransformerEncoder(
        intermediate_dim=512,
        num_heads=2,
        dropout=0.25
    )(sequence)

outputs = keras.layers.Dense(2)(sequence[:,backbone.cls_token_index,:])
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.AdamW(5e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True
)

model.fit(
    imdb_train_ds,
    validation_data=imdb_test_ds,
    epochs=1
)

100%|██████████| 507/507 [00:00<00:00, 892kB/s]


100%|██████████| 547/547 [00:00<00:00, 494kB/s]


100%|██████████| 226k/226k [00:00<00:00, 1.20MB/s]


100%|██████████| 16.8M/16.8M [00:00<00:00, 23.0MB/s]


625/625 ━━━━━━━━━━━━━━━━━━━━ 50s 43ms/step - loss: 0.6937 - sparse_categorical_accuracy: 0.5950 - val_loss: 0.5579 - val_sparse_categorical_accuracy: 0.7170
